In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
path = './'
train = pd.read_csv(path+'/input/train_set.csv')
test = pd.read_csv(path+'/input/test_set.csv')

In [2]:
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [3]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [4]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import lightgbm as lgb

//anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [19]:
test['y'] = -1

cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

feature1 = cate_features + num_features
data = pd.concat([train, test])

def feat_count(df, df_feature, fe, value, name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % (fe[0])]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left")#.fillna(0)
    return df, df_count.columns[1]

for value in feature1:
    data = feat_count(data, data, ['ID'], value)[0] #[0]是取函数结果的数据

data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_ID_count,marital_ID_count,education_ID_count,default_ID_count,housing_ID_count,loan_ID_count,contact_ID_count,month_ID_count,poutcome_ID_count,age_ID_count,balance_ID_count,day_ID_count,duration_ID_count,campaign_ID_count,pdays_ID_count,previous_ID_count
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [20]:
#该函数用来建立组合特征count计数的新特征
def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name}) #size取每种组合特征的个数
    data = data.merge(temp, 'left', on=features)
    return data, feature_name

# 建立交叉属性count特征
ll=[]
for f in ['campaign', 'contact', 'default', 'education', 'housing', 'job', 'loan', 'marital', 'poutcome', 'pdays', 'previous']:
    data, feature_name = feature_count(data, ['day', 'month', f])
    ll.append('_'.join(['count', 'day', 'month', f]))

print(ll)

data.head()

['count_day_month_campaign', 'count_day_month_contact', 'count_day_month_default', 'count_day_month_education', 'count_day_month_housing', 'count_day_month_job', 'count_day_month_loan', 'count_day_month_marital', 'count_day_month_poutcome', 'count_day_month_pdays', 'count_day_month_previous']


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_ID_count,marital_ID_count,education_ID_count,default_ID_count,housing_ID_count,loan_ID_count,contact_ID_count,month_ID_count,poutcome_ID_count,age_ID_count,balance_ID_count,day_ID_count,duration_ID_count,campaign_ID_count,pdays_ID_count,previous_ID_count,count_day_month_campaign,count_day_month_contact,count_day_month_default,count_day_month_education,count_day_month_housing,count_day_month_job,count_day_month_loan,count_day_month_marital,count_day_month_poutcome,count_day_month_pdays,count_day_month_previous
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,112,338,326,67,303,46,292,180,338,338,338
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,41,68,71,9,54,12,62,9,7,1,8
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,78,204,230,147,138,30,64,166,235,235,235
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,49,194,194,110,130,60,72,53,206,206,206
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,15,357,361,213,326,63,313,24,358,358,358


In [22]:
def get_new_columns(name, aggs):
    l = []
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg)) == "<class 'function'>":
                l.append(name + '_' + k + '_' + 'other')
            else:
                l.append(name + '_' + k + '_' + agg)
    return l

In [24]:
%%time
# 类别变量
sparse_feature = ['campaign', 'contact', 'default', 'education', 'housing', 'job', 'loan', 'marital', 'month', 'poutcome']
# 连续变量
dense_feature = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# 类别与类别变量交叉时统计count与nunique， 类别与连续变量交叉时统计均值，最大值，最小值，方差
for sf in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s] = ['count','nunique']
    for den in dense_feature:
        aggs[den] = ['mean', 'max', 'min', 'std']
    aggs.pop(sf) #组合特征时避免与自身组合
    temp = data.groupby(sf).agg(aggs).reset_index()
    temp.columns = [sf] + get_new_columns(sf, aggs)
    data = pd.merge(data, temp, on=sf, how='left')

100%|██████████| 10/10 [00:10<00:00,  1.65s/it]

CPU times: user 17.6 s, sys: 2.22 s, total: 19.8 s
Wall time: 10.5 s


In [25]:
df_train_columns = data.columns.tolist()
df_train_columns

['ID',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y',
 'job_ID_count',
 'marital_ID_count',
 'education_ID_count',
 'default_ID_count',
 'housing_ID_count',
 'loan_ID_count',
 'contact_ID_count',
 'month_ID_count',
 'poutcome_ID_count',
 'age_ID_count',
 'balance_ID_count',
 'day_ID_count',
 'duration_ID_count',
 'campaign_ID_count',
 'pdays_ID_count',
 'previous_ID_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nunique',
 'campaig

In [31]:
num_features = [col for col in data.select_dtypes(exclude='object').columns if col not in ['ID', 'y']]

In [32]:
num_features

['job_ID_count',
 'marital_ID_count',
 'education_ID_count',
 'default_ID_count',
 'housing_ID_count',
 'loan_ID_count',
 'contact_ID_count',
 'month_ID_count',
 'poutcome_ID_count',
 'age_ID_count',
 'balance_ID_count',
 'day_ID_count',
 'duration_ID_count',
 'campaign_ID_count',
 'pdays_ID_count',
 'previous_ID_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nunique',
 'campaign_housing_count',
 'campaign_housing_nunique',
 'campaign_job_count',
 'campaign_job_nunique',
 'campaign_loan_count',
 'campaign_loan_nunique',
 'campaign_marital_count',
 'campaign_marital_n

In [36]:
%%time
#没用'ID',
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.model_selection import KFold


one_hot_feature = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


# def encode_count(df,column_name):
#     lbl = preprocessing.LabelEncoder()
#     lbl.fit(list(df[column_name].values))
#     df[column_name] = lbl.transform(list(df[column_name].values))
#     return df

data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact', 'month', 'poutcome']:
    data['new_con'] = data['new_con'].astype(str) + ' ' + data[i].astype(str)
#data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))





for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
        
vector_feature = ['new_con']

train = data[data.y != -1]
test = data[data.y == -1]
# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
res = test[['ID']]
# test=test.drop(['ID'],axis=1)

#先取出数值特征，类别特征做独热处理
train_x = train[num_features]
test_x = test[num_features]

enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a = enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

# ['new_con']为空格分开的字符串，对其用cv得到更多的特征
cv = CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')


CPU times: user 23.3 s, sys: 778 ms, total: 24.1 s
Wall time: 4.78 s


In [66]:
%%time
# trainxx, vali_x, trainyy, vali_y = train_test_split(train_x, train_y, test_size=0.2, random_state=2018)

# colsample_bytree生成树时的列采样
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_y = train['y']
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx, :]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x[val_idx, :]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1),(test_x1, test_y1)], eval_metric='auc', early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:, 1]

res['pred'] = res['pred'] / n_splits
res.to_csv('./basesubmission.csv', index=False)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


[1]	valid_0's auc: 0.778786	valid_1's auc: 0.797897
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.923058	valid_1's auc: 0.91483
[3]	valid_0's auc: 0.915587	valid_1's auc: 0.90962
[4]	valid_0's auc: 0.931177	valid_1's auc: 0.92355
[5]	valid_0's auc: 0.935278	valid_1's auc: 0.926294
[6]	valid_0's auc: 0.933318	valid_1's auc: 0.924534
[7]	valid_0's auc: 0.931453	valid_1's auc: 0.923929
[8]	valid_0's auc: 0.928863	valid_1's auc: 0.922126
[9]	valid_0's auc: 0.933352	valid_1's auc: 0.925733
[10]	valid_0's auc: 0.936455	valid_1's auc: 0.930593
[11]	valid_0's auc: 0.934911	valid_1's auc: 0.929134
[12]	valid_0's auc: 0.933616	valid_1's auc: 0.927798
[13]	valid_0's auc: 0.931978	valid_1's auc: 0.92618
[14]	valid_0's auc: 0.934734	valid_1's auc: 0.928241
[15]	valid_0's auc: 0.933387	valid_1's auc: 0.927016
[16]	valid_0's auc: 0.935813	valid_1's auc: 0.928973
[17]	valid_0's auc: 0.935066	valid_1's auc: 0.9277
[18]	valid_0's auc: 0.937302	valid_1's auc: 0.93012

[179]	valid_0's auc: 0.962648	valid_1's auc: 0.939444
[180]	valid_0's auc: 0.96273	valid_1's auc: 0.93948
[181]	valid_0's auc: 0.962804	valid_1's auc: 0.939439
[182]	valid_0's auc: 0.962869	valid_1's auc: 0.939402
[183]	valid_0's auc: 0.962958	valid_1's auc: 0.939391
[184]	valid_0's auc: 0.963059	valid_1's auc: 0.939413
[185]	valid_0's auc: 0.963118	valid_1's auc: 0.939453
[186]	valid_0's auc: 0.963189	valid_1's auc: 0.939429
[187]	valid_0's auc: 0.963275	valid_1's auc: 0.939477
[188]	valid_0's auc: 0.963361	valid_1's auc: 0.939536
[189]	valid_0's auc: 0.963446	valid_1's auc: 0.939492
[190]	valid_0's auc: 0.963524	valid_1's auc: 0.939506
[191]	valid_0's auc: 0.963629	valid_1's auc: 0.939553
[192]	valid_0's auc: 0.963725	valid_1's auc: 0.939567
[193]	valid_0's auc: 0.963815	valid_1's auc: 0.939564
[194]	valid_0's auc: 0.963921	valid_1's auc: 0.939558
[195]	valid_0's auc: 0.964006	valid_1's auc: 0.939538
[196]	valid_0's auc: 0.964083	valid_1's auc: 0.939562
[197]	valid_0's auc: 0.964161	

[348]	valid_0's auc: 0.975314	valid_1's auc: 0.940517
[349]	valid_0's auc: 0.975366	valid_1's auc: 0.940498
[350]	valid_0's auc: 0.975426	valid_1's auc: 0.940478
[351]	valid_0's auc: 0.975485	valid_1's auc: 0.940515
[352]	valid_0's auc: 0.975542	valid_1's auc: 0.940545
[353]	valid_0's auc: 0.975601	valid_1's auc: 0.940519
[354]	valid_0's auc: 0.975675	valid_1's auc: 0.940495
[355]	valid_0's auc: 0.97575	valid_1's auc: 0.940464
[356]	valid_0's auc: 0.975796	valid_1's auc: 0.94045
[357]	valid_0's auc: 0.975861	valid_1's auc: 0.940437
[358]	valid_0's auc: 0.975935	valid_1's auc: 0.94045
[359]	valid_0's auc: 0.975994	valid_1's auc: 0.940481
[360]	valid_0's auc: 0.976054	valid_1's auc: 0.94047
[361]	valid_0's auc: 0.9761	valid_1's auc: 0.940478
[362]	valid_0's auc: 0.976162	valid_1's auc: 0.940485
[363]	valid_0's auc: 0.976217	valid_1's auc: 0.940484
[364]	valid_0's auc: 0.976304	valid_1's auc: 0.940459
[365]	valid_0's auc: 0.976357	valid_1's auc: 0.940468
[366]	valid_0's auc: 0.976402	vali

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

[186]	valid_0's auc: 0.962581	valid_1's auc: 0.943298
[187]	valid_0's auc: 0.962659	valid_1's auc: 0.943363
[188]	valid_0's auc: 0.962787	valid_1's auc: 0.943348
[189]	valid_0's auc: 0.962847	valid_1's auc: 0.94334
[190]	valid_0's auc: 0.96294	valid_1's auc: 0.943323
[191]	valid_0's auc: 0.963001	valid_1's auc: 0.943331
[192]	valid_0's auc: 0.96311	valid_1's auc: 0.943376
[193]	valid_0's auc: 0.963228	valid_1's auc: 0.943362
[194]	valid_0's auc: 0.963322	valid_1's auc: 0.943369
[195]	valid_0's auc: 0.963421	valid_1's auc: 0.943421
[196]	valid_0's auc: 0.963549	valid_1's auc: 0.943407
[197]	valid_0's auc: 0.963655	valid_1's auc: 0.943409
[198]	valid_0's auc: 0.963739	valid_1's auc: 0.943413
[199]	valid_0's auc: 0.96383	valid_1's auc: 0.943446
[200]	valid_0's auc: 0.963949	valid_1's auc: 0.94344
[201]	valid_0's auc: 0.964051	valid_1's auc: 0.943514
[202]	valid_0's auc: 0.964109	valid_1's auc: 0.943497
[203]	valid_0's auc: 0.964206	valid_1's auc: 0.943516
[204]	valid_0's auc: 0.964276	val

[339]	valid_0's auc: 0.974737	valid_1's auc: 0.944258
[340]	valid_0's auc: 0.974797	valid_1's auc: 0.944279
[341]	valid_0's auc: 0.974837	valid_1's auc: 0.944247
[342]	valid_0's auc: 0.974894	valid_1's auc: 0.944247
[343]	valid_0's auc: 0.974958	valid_1's auc: 0.94426
[344]	valid_0's auc: 0.975021	valid_1's auc: 0.94426
[345]	valid_0's auc: 0.975085	valid_1's auc: 0.944302
[346]	valid_0's auc: 0.975143	valid_1's auc: 0.944312
[347]	valid_0's auc: 0.975221	valid_1's auc: 0.944329
[348]	valid_0's auc: 0.975286	valid_1's auc: 0.944295
[349]	valid_0's auc: 0.975358	valid_1's auc: 0.944315
[350]	valid_0's auc: 0.975419	valid_1's auc: 0.944302
[351]	valid_0's auc: 0.975467	valid_1's auc: 0.944251
[352]	valid_0's auc: 0.975528	valid_1's auc: 0.944283
[353]	valid_0's auc: 0.975588	valid_1's auc: 0.944248
[354]	valid_0's auc: 0.975654	valid_1's auc: 0.944275
[355]	valid_0's auc: 0.975714	valid_1's auc: 0.944281
[356]	valid_0's auc: 0.975789	valid_1's auc: 0.94427
[357]	valid_0's auc: 0.975839	v

[38]	valid_0's auc: 0.947388	valid_1's auc: 0.930507
[39]	valid_0's auc: 0.947503	valid_1's auc: 0.930778
[40]	valid_0's auc: 0.947697	valid_1's auc: 0.931088
[41]	valid_0's auc: 0.947831	valid_1's auc: 0.931208
[42]	valid_0's auc: 0.947932	valid_1's auc: 0.931208
[43]	valid_0's auc: 0.948142	valid_1's auc: 0.931197
[44]	valid_0's auc: 0.948279	valid_1's auc: 0.931142
[45]	valid_0's auc: 0.948465	valid_1's auc: 0.931307
[46]	valid_0's auc: 0.948654	valid_1's auc: 0.931461
[47]	valid_0's auc: 0.94879	valid_1's auc: 0.931559
[48]	valid_0's auc: 0.948892	valid_1's auc: 0.931599
[49]	valid_0's auc: 0.949025	valid_1's auc: 0.931677
[50]	valid_0's auc: 0.949154	valid_1's auc: 0.931706
[51]	valid_0's auc: 0.949336	valid_1's auc: 0.931713
[52]	valid_0's auc: 0.949569	valid_1's auc: 0.931782
[53]	valid_0's auc: 0.949723	valid_1's auc: 0.931856
[54]	valid_0's auc: 0.94983	valid_1's auc: 0.931883
[55]	valid_0's auc: 0.949931	valid_1's auc: 0.931977
[56]	valid_0's auc: 0.950029	valid_1's auc: 0.93

[195]	valid_0's auc: 0.965912	valid_1's auc: 0.934328
[196]	valid_0's auc: 0.965998	valid_1's auc: 0.934296
[197]	valid_0's auc: 0.966048	valid_1's auc: 0.934306
[198]	valid_0's auc: 0.966114	valid_1's auc: 0.934356
[199]	valid_0's auc: 0.966173	valid_1's auc: 0.934358
[200]	valid_0's auc: 0.966262	valid_1's auc: 0.934386
[201]	valid_0's auc: 0.96636	valid_1's auc: 0.934396
[202]	valid_0's auc: 0.966434	valid_1's auc: 0.934412
[203]	valid_0's auc: 0.966529	valid_1's auc: 0.934413
[204]	valid_0's auc: 0.966604	valid_1's auc: 0.934385
[205]	valid_0's auc: 0.966669	valid_1's auc: 0.934364
[206]	valid_0's auc: 0.966749	valid_1's auc: 0.93431
[207]	valid_0's auc: 0.96687	valid_1's auc: 0.934277
[208]	valid_0's auc: 0.966938	valid_1's auc: 0.934251
[209]	valid_0's auc: 0.96702	valid_1's auc: 0.93429
[210]	valid_0's auc: 0.967081	valid_1's auc: 0.934269
[211]	valid_0's auc: 0.96713	valid_1's auc: 0.934272
[212]	valid_0's auc: 0.967206	valid_1's auc: 0.934261
[213]	valid_0's auc: 0.967299	vali

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
